# Imports and Utilities

In [1]:
from typing import List, Tuple
import numpy as np
import pandas as pd
from IPython.display import Markdown
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.model_selection import cross_val_score

COMPETITION_NAME = "warmup-round-march-machine-learning-mania-2023"
VERBOSE = False

pd.set_option("display.max_columns", None)

def show_df(df: pd.DataFrame, name: str = "DataFrame", verbose: bool = VERBOSE):
    if verbose:
        display(Markdown(f"## {name}"))
        display(Markdown("### Head"))
        display(df.head())
        display(Markdown("### Tail"))
        display(df.tail())
        display(Markdown("### Description"))
        display(df.describe())


def extract_data(filename: str, competition_name:str = COMPETITION_NAME) -> pd.DataFrame:
    mens_filepath = f"/kaggle/input/{competition_name}/M{filename}.csv"
    womens_filepath = f"/kaggle/input/{competition_name}/W{filename}.csv"
    try:
        df_mens = pd.read_csv(mens_filepath)
    except FileNotFoundError:
        df_mens = None
    try:
        df_womens = pd.read_csv(womens_filepath)
    except FileNotFoundError:
        df_womens = None
    df = pd.concat([df_mens, df_womens])
    show_df(df, filename)
    return df


def get_team_features(detailed_results: pd.DataFrame) -> pd.DataFrame:
    df = detailed_results.copy()
    df = clean_detailed_results(df)
    df = transform_game_to_team(df)
    df = transform_team_results(df)
    show_df(df)
    return df


def get_seed_features(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    mask = df["Season"] > 2002
    df = df[mask]
    df["Seed"] = df["Seed"].str.replace(r"\D+","", regex=True)
    df["Seed"] = df["Seed"].astype(int)
    show_df(df)
    return df


def get_ranking_features(df_in:pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    mask = df["RankingDayNum"] == df["RankingDayNum"].max()
    df = df[mask]
    df.drop(["SystemName", "RankingDayNum"], axis=1, inplace=True)
    df = df.groupby(["Season", "TeamID"]).agg("median")
    df = df.reset_index()
    show_df(df)
    return df


def get_game_outcomes(df):
    input_rows = df.to_records()
    output_rows = [parse_row(input_row) for input_row in input_rows]
    df = pd.DataFrame(output_rows)
    show_df(df)
    return df


def clean_detailed_results(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(["WLoc", "DayNum"], axis=1)


def transform_game_to_team(game_results: pd.DataFrame) -> pd.DataFrame:
    winners = rename_columns(game_results, "W")
    loosers = rename_columns(game_results, "L")
    team_results = pd.concat((winners, loosers))
    team_results.drop(["TeamIDOpp"], axis=1, inplace=True)
    return team_results


def transform_team_results(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby(["Season", "TeamID"]).median()
    df["FGP"] =  df["FGM"] / df["FGA"]
    df["FGP3"] =  df["FGM3"] / df["FGA3"]
    df["FTP"] =  df["FTM"] / df["FTA"]
    df["FGPOpp"] =  df["FGMOpp"] / df["FGAOpp"]
    df["FGP3Opp"] =  df["FGM3Opp"] / df["FGA3Opp"]
    df["FTPOpp"] =  df["FTMOpp"] / df["FTAOpp"]
    return df.reset_index()
    
    
def rename_columns(df: pd.DataFrame, team_prefix: str) -> pd.DataFrame:
    df = df.copy()
    df.columns =  (rename_column(column_name, team_prefix) for column_name in df.columns)
    return df


def rename_column(column_name: str, team_prefix: str) -> pd.DataFrame:
    if team_prefix == "W":
        opponent_prefix = "L"
    elif team_prefix == "L":
        opponent_prefix = "W"
    else:
        raise ValueError
    if column_name.startswith(team_prefix):
        column_name = column_name.lstrip(team_prefix)
    elif column_name.startswith(opponent_prefix):
        column_name = f"{column_name.lstrip(opponent_prefix)}Opp"
    return column_name


def split_winner_and_looser_columns(df: pd.DataFrame) -> Tuple[List[str], List[str]]:
    winner_columns = [name for name in df.columns if not name.startswith("L")]
    looser_columns = [name for name in df.columns if not name.startswith("W")]
    return winner_columns, looser_columns


def merge_features(
    season_features: pd.DataFrame, 
    tournament_features: pd.DataFrame, 
    seed_features: pd.DataFrame, 
    ranking_features: pd.DataFrame
) -> pd.DataFrame:
    features = pd.merge(
        season_features,
        tournament_features,
        how="inner",
        on=["Season", "TeamID"],
        suffixes=("Reg", "Tou")
    )
    features = features.merge(
        seed_features,
        how="inner",
        on=["Season", "TeamID"]
    )
    features = features.merge(
        ranking_features,
        how="left",
        on=["Season", "TeamID"],
    )
    show_df(features)
    return features


def parse_row(row):
    season = row['Season']
    winning_team_id = row['WTeamID']
    losing_team_id = row['LTeamID']
    if winning_team_id < losing_team_id:
        small_id = winning_team_id
        big_id = losing_team_id
        outcome = True
    elif losing_team_id < winning_team_id:
        small_id = losing_team_id
        big_id = winning_team_id
        outcome = False
    record = {
            "ID": f"{season}_{small_id}_{big_id}",
            'Season': season,
            'LowID': small_id,
            'HighID': big_id,
            'Win': outcome
    }
    return record


def merge_outcomes_with_features(outcomes: pd.DataFrame, features: pd.DataFrame, how: str = "inner") -> pd.DataFrame:
    feature_names = [name for name in features.columns if name not in ["Season", "TeamID", "Gender"]]
    data = pd.merge(
        outcomes, 
        features, 
        how=how, 
        left_on=["Season", "HighID"], 
        right_on=["Season", "TeamID"]
    )
    data = pd.merge(
        data, 
        features, 
        how=how, 
        left_on=["Season", "LowID"], 
        right_on=["Season", "TeamID"],
        suffixes=("High", "Low")
    )
    for name in feature_names:
        data[f"{name}Diff"] = data[f"{name}High"] - data[f"{name}Low"]
        data.drop([f"{name}High", f"{name}Low"], axis=1, inplace=True)
    data.drop(
        ["Season", "HighID", "LowID","TeamIDHigh","TeamIDLow"], 
        axis=1, 
        inplace=True
    )
    data.set_index("ID", inplace=True)
    show_df(data)
    return data


def get_submission_outcomes() -> pd.DataFrame:
    sample_submission = pd.read_csv(f"/kaggle/input/{COMPETITION_NAME}/SampleSubmissionWarmup.csv")
    df = sample_submission.copy()
    df.drop("Pred", axis=1, inplace=True)
    df[["Season", "LowID", "HighID"]] = df["ID"].str.split("_", expand=True)
    df[["Season", "LowID", "HighID"]] = df[["Season", "LowID", "HighID"]].astype(int)
    show_df(df)
    return df


def objective(trial: optuna.Trial, X_train, y_train):
    params = {
        "objective": "binary",
        "metric": "l2",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    model = lgb.LGBMClassifier(**params)
    score = np.mean(cross_val_score(model, X_train, y_train, scoring="neg_brier_score", cv=5))
    return score


def run_study(X_train, y_train):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100)
    return study


def train_optimized_model(study, X, y):
    params = study.best_params
    model = lgb.LGBMClassifier(
        objective="binary",
        metric="l2",
        verbosity=-1,
        boosting_type="gbdt",
        **params
    )
    model = model.fit(X, y)
    return model

# Load Data

In [2]:
season_results = extract_data("RegularSeasonDetailedResults")
tournament_results = extract_data("NCAATourneyDetailedResults")
seeds = extract_data("NCAATourneySeeds")
rankings = extract_data("MasseyOrdinals")

# Transform Data

In [3]:
season_team_features = get_team_features(season_results)

tournament_results["Season"] += 1
tournament_team_features = get_team_features(tournament_results)

seed_features = get_seed_features(seeds)

ranking_features = get_ranking_features(rankings)

## Merge features

In [4]:
features = merge_features(season_team_features, tournament_team_features, seed_features, ranking_features)

## Build Datasets

In [5]:
from sklearn.model_selection import train_test_split

data = extract_data("NCAATourneyCompactResults")
data_train, data_valid = train_test_split(data, random_state=0)

outcomes = get_game_outcomes(data)
outcomes_train = get_game_outcomes(data_train)
outcomes_valid = get_game_outcomes(data_valid)
outcomes_submission = get_submission_outcomes()

In [6]:
features_train = merge_outcomes_with_features(outcomes_train, features)
features_valid = merge_outcomes_with_features(outcomes_valid, features)
features_full = merge_outcomes_with_features(outcomes, features)

In [7]:
y_train = features_train["Win"]
X_train = features_train.drop("Win", axis=1)
y_valid = features_valid["Win"]
X_valid = features_valid.drop("Win", axis=1)
y = features_full["Win"]
X = features_full.drop("Win", axis=1)
X_submission = merge_outcomes_with_features(outcomes_submission, features, how="left").fillna(0)

# Step 4: Train a model


In [8]:
study = run_study(X_train, y_train)
study.best_params

[I 2023-03-15 16:47:26,607] A new study created in memory with name: no-name-2eac46fd-d02b-490d-9472-1014a48b800a


[LightGBM] [Warning] bagging_fraction is set=0.8737065008476477, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8737065008476477
[LightGBM] [Warning] lambda_l1 is set=9.971306902267015e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.971306902267015e-08
[LightGBM] [Warning] feature_fraction is set=0.6394715984948618, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6394715984948618
[LightGBM] [Warning] lambda_l2 is set=0.0003128122059383428, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0003128122059383428
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8737065008476477, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8737065008476477
[LightGBM] [Warning] lambda_l1 is set=9.971306902267015e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.971306902267015e-08
[LightGBM] [Warning] featu

[I 2023-03-15 16:47:26,978] Trial 0 finished with value: -0.21994829304718316 and parameters: {'lambda_l1': 9.971306902267015e-08, 'lambda_l2': 0.0003128122059383428, 'num_leaves': 24, 'feature_fraction': 0.6394715984948618, 'bagging_fraction': 0.8737065008476477, 'bagging_freq': 3, 'min_child_samples': 31}. Best is trial 0 with value: -0.21994829304718316.


[LightGBM] [Warning] bagging_fraction is set=0.8737065008476477, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8737065008476477
[LightGBM] [Warning] lambda_l1 is set=9.971306902267015e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.971306902267015e-08
[LightGBM] [Warning] feature_fraction is set=0.6394715984948618, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6394715984948618
[LightGBM] [Warning] lambda_l2 is set=0.0003128122059383428, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0003128122059383428
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8737065008476477, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8737065008476477
[LightGBM] [Warning] lambda_l1 is set=9.971306902267015e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.971306902267015e-08
[LightGBM] [Warning] featu

[I 2023-03-15 16:47:27,202] Trial 1 finished with value: -0.2044357163180759 and parameters: {'lambda_l1': 0.8288560347882109, 'lambda_l2': 0.0007511806272907411, 'num_leaves': 229, 'feature_fraction': 0.555111349452285, 'bagging_fraction': 0.47676260301270473, 'bagging_freq': 1, 'min_child_samples': 39}. Best is trial 1 with value: -0.2044357163180759.


...

[I 2023-03-15 16:47:51,052] Trial 97 finished with value: -0.1905029712496248 and parameters: {'lambda_l1': 4.090666035787813, 'lambda_l2': 1.8450423075745337e-06, 'num_leaves': 144, 'feature_fraction': 0.5481034068753673, 'bagging_fraction': 0.8636241938834719, 'bagging_freq': 4, 'min_child_samples': 83}. Best is trial 33 with value: -0.18757557690127297.


[LightGBM] [Warning] bagging_fraction is set=0.8636241938834719, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8636241938834719
[LightGBM] [Warning] lambda_l1 is set=4.090666035787813, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.090666035787813
[LightGBM] [Warning] feature_fraction is set=0.5481034068753673, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5481034068753673
[LightGBM] [Warning] lambda_l2 is set=1.8450423075745337e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8450423075745337e-06
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_fraction is set=0.8636241938834719, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8636241938834719
[LightGBM] [Warning] lambda_l1 is set=4.090666035787813, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.090666035787813
[LightGBM] [Warning] feature_fraction is

[I 2023-03-15 16:47:51,437] Trial 98 finished with value: -0.20977431770717447 and parameters: {'lambda_l1': 2.1057503044370467, 'lambda_l2': 6.532556519606388e-08, 'num_leaves': 104, 'feature_fraction': 0.5305437036193085, 'bagging_fraction': 0.8257022369051376, 'bagging_freq': 3, 'min_child_samples': 18}. Best is trial 33 with value: -0.18757557690127297.
[I 2023-03-15 16:47:51,655] Trial 99 finished with value: -0.19531425943472164 and parameters: {'lambda_l1': 0.7474601890471756, 'lambda_l2': 7.421180366306739e-07, 'num_leaves': 133, 'feature_fraction': 0.5085545239313585, 'bagging_fraction': 0.7911891770806574, 'bagging_freq': 5, 'min_child_samples': 88}. Best is trial 33 with value: -0.18757557690127297.


[LightGBM] [Warning] bagging_fraction is set=0.7911891770806574, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7911891770806574
[LightGBM] [Warning] lambda_l1 is set=0.7474601890471756, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7474601890471756
[LightGBM] [Warning] feature_fraction is set=0.5085545239313585, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5085545239313585
[LightGBM] [Warning] lambda_l2 is set=7.421180366306739e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.421180366306739e-07
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.7911891770806574, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7911891770806574
[LightGBM] [Warning] lambda_l1 is set=0.7474601890471756, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7474601890471756
[LightGBM] [Warning] feature_fraction 

{'lambda_l1': 8.42578768681397,
 'lambda_l2': 2.4201088034013655e-06,
 'num_leaves': 27,
 'feature_fraction': 0.5536963500833583,
 'bagging_fraction': 0.7844954777732736,
 'bagging_freq': 4,
 'min_child_samples': 81}

In [9]:
model = train_optimized_model(study, X_train, y_train)
preds = model.predict_proba(X_valid)[:,1]
print("Brier Score:", brier_score_loss(y_valid, preds))
print("ROC AUC:", roc_auc_score(y_valid, preds))
print("Parameters")
print(*(f"- {key}: {value}" for key, value in model.get_params(deep=True).items()), sep="\n")
print("Features")
print(*(f"- {name}: {imp}" for name, imp in sorted(zip(model.feature_name_, model.feature_importances_), key=lambda x: x[1], reverse=True)), sep="\n")


[LightGBM] [Warning] bagging_fraction is set=0.7844954777732736, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7844954777732736
[LightGBM] [Warning] lambda_l1 is set=8.42578768681397, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.42578768681397
[LightGBM] [Warning] feature_fraction is set=0.5536963500833583, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5536963500833583
[LightGBM] [Warning] lambda_l2 is set=2.4201088034013655e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.4201088034013655e-06
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Brier Score: 0.19014310675875265
ROC AUC: 0.7866698518872433
Parameters
- boosting_type: gbdt
- class_weight: None
- colsample_bytree: 1.0
- importance_type: split
- learning_rate: 0.1
- max_depth: -1
- min_child_samples: 81
- min_child_weight: 0.001
- min_split_gain: 0.0
- n_estimators: 100
- n_jobs: -1
- num_leaves: 

# Submit to the competition

We"ll begin by using the trained model to generate predictions, which we"ll save to a CSV file.

In [10]:
# Use the model to generate predictions
model.fit(X, y)
predictions = model.predict_proba(X_submission)[:,1]

# Save the predictions to a CSV file
output = pd.DataFrame({"ID": X_submission.index,
                       "Pred": predictions})
output.to_csv("submission.csv", index=False)
print(output.shape)
output.describe()

[LightGBM] [Warning] bagging_fraction is set=0.7844954777732736, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7844954777732736
[LightGBM] [Warning] lambda_l1 is set=8.42578768681397, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.42578768681397
[LightGBM] [Warning] feature_fraction is set=0.5536963500833583, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5536963500833583
[LightGBM] [Warning] lambda_l2 is set=2.4201088034013655e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.4201088034013655e-06
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
(614319, 2)


,Pred
count,614319.000000
mean,0.648322
std,0.023777
min,0.066631
25%,0.649453
50%,0.649453
75%,0.649453
max,0.949437
